In [ ]:
!pip install torch torchvision
!pip install matplotlib
!pip install pillow


In [ ]:
import torch
import torch.optim as optim
from torch import nn
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving generated_image.png to generated_image (1).png
Saving unsplash1.jpg to unsplash1.jpg


In [ ]:
def load_image(image_path, max_size=512, shape=None):
    image = Image.open(image_path).convert('RGB')
    if max(image.size) > max_size:
        size = max_size
    else:
        size = max(image.size)
    if shape:
        size = shape
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    image = transform(image).unsqueeze(0)
    return image

content_image = load_image('generated_image.png')
style_image = load_image('unsplash1.jpg', shape=content_image.shape[2:])


In [ ]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.vgg = models.vgg19(pretrained=True).features.eval()

    def forward(self, x):
        layers = [0, 5, 10, 19, 28]  # indices of the layers where we want to extract features
        features = []
        for i, layer in enumerate(self.vgg):
            x = layer(x)
            if i in layers:
                features.append(x)
        return features

def get_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features

def compute_loss(content_weight, style_weight, content_features, style_features, target_features):
    content_loss = torch.mean((target_features[0] - content_features[0]) ** 2)
    style_loss = 0
    for target_feature, style_feature in zip(target_features[1:], style_features):
        target_gram = gram_matrix(target_feature)
        style_gram = gram_matrix(style_feature)
        style_loss += torch.mean((target_gram - style_gram) ** 2)

    content_loss *= content_weight
    style_loss *= style_weight
    return content_loss + style_loss

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    G = torch.mm(features, features.t())
    return G.div(c * h * w)


In [ ]:
from google.colab import files
import torch
from torchvision import transforms
from PIL import Image

# Upload files
uploaded = files.upload()

# Load image function
def load_image(image_path, max_size=512, shape=None):
    image = Image.open(image_path).convert('RGB')
    if max(image.size) > max_size:
        size = max_size
    else:
        size = max(image.size)
    if shape:
        size = shape
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    image = transform(image).unsqueeze(0)
    return image

# Replace with your actual uploaded image filenames
content_image = load_image('generated_image.png')  # Content image filename
style_image = load_image('unsplash1.jpg', shape=content_image.shape[2:])  # Style image filename


Saving generated_image.png to generated_image.png
Saving unsplash1.jpg to unsplash1.jpg


In [ ]:
import torch
import torch.optim as optim
from torch import nn
from torchvision import models

# Define VGG19 model
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.vgg = models.vgg19(pretrained=True).features.eval()

    def forward(self, x):
        layers = [0, 5, 10, 19, 28]
        features = []
        for i, layer in enumerate(self.vgg):
            x = layer(x)
            if i in layers:
                features.append(x)
        return features

def get_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features

def compute_loss(content_weight, style_weight, content_features, style_features, target_features):
    content_loss = torch.mean((target_features[0] - content_features[0]) ** 2)
    style_loss = 0
    for target_feature, style_feature in zip(target_features[1:], style_features):
        target_gram = gram_matrix(target_feature)
        style_gram = gram_matrix(style_feature)
        style_loss += torch.mean((target_gram - style_gram) ** 2)

    content_loss *= content_weight
    style_loss *= style_weight
    return content_loss + style_loss

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    G = torch.mm(features, features.t())
    return G.div(c * h * w)

def run_style_transfer(content_img, style_img, num_steps=300, content_weight=1, style_weight=1000000):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VGG19().to(device)
    content_features = get_features(model, content_img.to(device))
    style_features = get_features(model, style_img.to(device))

    target = content_img.clone().requires_grad_(True).to(device)
    optimizer = optim.LBFGS([target])

    for step in range(num_steps):
        def closure():
            optimizer.zero_grad()
            target_features = get_features(model, target)
            loss = compute_loss(content_weight, style_weight, content_features, style_features, target_features)
            loss.backward()
            if step % 50 == 0:
                print(f"Step {step}, Loss {loss.item()}")
            return loss

        optimizer.step(closure)

    return target


In [ ]:
def run_style_transfer(content_img, style_img, num_steps=300, content_weight=1, style_weight=1000000):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VGG19().to(device)
    content_features = get_features(model, content_img.to(device))
    style_features = get_features(model, style_img.to(device))

    # Detach the target from the computation graph to make it a leaf tensor
    target = content_img.clone().detach().requires_grad_(True).to(device)

    optimizer = optim.LBFGS([target])

    for step in range(num_steps):
        def closure():
            optimizer.zero_grad()
            target_features = get_features(model, target)
            loss = compute_loss(content_weight, style_weight, content_features, style_features, target_features)
            loss.backward()
            if step % 50 == 0:
                print(f"Step {step}, Loss {loss.item()}")
            return loss

        optimizer.step(closure)

    return target

In [ ]:
# Perform style transfer
output_img = run_style_transfer(content_image, style_image)

# Convert output image back to PIL format
def tensor_to_image(tensor):
    tensor = tensor.squeeze().cpu().detach().clamp(0, 1)
    tensor = transforms.ToPILImage()(tensor)
    return tensor

output_pil_image = tensor_to_image(output_img)
plt.imshow(output_pil_image)
plt.axis('off')
plt.show()

# Save or download the result
output_pil_image.save('output_image.png')
files.download('output_image.png')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: can't optimize a non-leaf Tensor

In [ ]:
import torch
import torch.optim as optim
from torch import nn
from torchvision import models

class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.vgg = models.vgg19(pretrained=True).features.eval()

    def forward(self, x):
        layers = [0, 5, 10, 19, 28]
        features = []
        for i, layer in enumerate(self.vgg):
            x = layer(x)
            if i in layers:
                features.append(x)
        return features

def get_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features

def compute_loss(content_weight, style_weight, content_features, style_features, target_features):
    content_loss = torch.mean((target_features[0] - content_features[0]) ** 2)
    style_loss = 0
    for target_feature, style_feature in zip(target_features[1:], style_features):
        target_gram = gram_matrix(target_feature)
        style_gram = gram_matrix(style_feature)
        style_loss += torch.mean((target_gram - style_gram) ** 2)

    content_loss *= content_weight
    style_loss *= style_weight
    return content_loss + style_loss

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    G = torch.mm(features, features.t())
    return G.div(c * h * w)

def run_style_transfer(content_img, style_img, num_steps=300, content_weight=1, style_weight=1000000):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VGG19().to(device)
    content_features = get_features(model, content_img.to(device))
    style_features = get_features(model, style_img.to(device))

    target = content_img.clone().requires_grad_(True).to(device)
    optimizer = optim.LBFGS([target])  # Make sure target is a leaf tensor

    for step in range(num_steps):
        def closure():
            optimizer.zero_grad()
            target_features = get_features(model, target)
            loss = compute_loss(content_weight, style_weight, content_features, style_features, target_features)
            loss.backward()
            if step % 50 == 0:
                print(f"Step {step}, Loss {loss.item()}")
            return loss

        optimizer.step(closure)

    return target

# Perform style transfer
output_img = run_style_transfer(content_image, style_image)

# Convert output image back to PIL format
def tensor_to_image(tensor):
    tensor = tensor.squeeze().cpu().detach().clamp(0, 1)
    tensor = transforms.ToPILImage()(tensor)
    return tensor

output_pil_image = tensor_to_image(output_img)
plt.imshow(output_pil_image)
plt.axis('off')
plt.show()

# Save or download the result
output_pil_image.save('output_image.png')
files.download('output_image.png')


ValueError: can't optimize a non-leaf Tensor

In [ ]:
import torch
import torch.optim as optim
from torch import nn
from torchvision import models

# Define the VGG19 model and related functions
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.vgg = models.vgg19(pretrained=True).features.eval()

    def forward(self, x):
        layers = [0, 5, 10, 19, 28]
        features = []
        for i, layer in enumerate(self.vgg):
            x = layer(x)
            if i in layers:
                features.append(x)
        return features

def get_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features

def compute_loss(content_weight, style_weight, content_features, style_features, target_features):
    content_loss = torch.mean((target_features[0] - content_features[0]) ** 2)
    style_loss = 0
    for target_feature, style_feature in zip(target_features[1:], style_features):
        target_gram = gram_matrix(target_feature)
        style_gram = gram_matrix(style_feature)
        style_loss += torch.mean((target_gram - style_gram) ** 2)

    content_loss *= content_weight
    style_loss *= style_weight
    return content_loss + style_loss

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(c, h * w)
    G = torch.mm(features, features.t())
    return G.div(c * h * w)

def run_style_transfer(content_img, style_img, num_steps=300, content_weight=1, style_weight=1000000):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VGG19().to(device)
    content_features = get_features(model, content_img.to(device))
    style_features = get_features(model, style_img.to(device))

    # Ensure that target is a leaf tensor
    target = content_img.clone().requires_grad_(True).to(device)

    # Use a different optimizer if LBFGS is not suitable
    optimizer = optim.Adam([target], lr=0.01)

    for step in range(num_steps):
        def closure():
            optimizer.zero_grad()
            target_features = get_features(model, target)
            loss = compute_loss(content_weight, style_weight, content_features, style_features, target_features)
            loss.backward()
            if step % 50 == 0:
                print(f"Step {step}, Loss {loss.item()}")
            return loss

        optimizer.step(closure)

    return target


In [ ]:
def run_style_transfer(content_img, style_img, num_steps=300, content_weight=1, style_weight=1000000):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VGG19().to(device)
    content_features = get_features(model, content_img.to(device))
    style_features = get_features(model, style_img.to(device))

    # Ensure that target is a leaf tensor
    target = content_img.clone().to(device)
    target.requires_grad_(True)

    optimizer = optim.LBFGS([target])

    for step in range(num_steps):
        def closure():
            optimizer.zero_grad()
            target_features = get_features(model, target)
            loss = compute_loss(content_weight, style_weight, content_features, style_features, target_features)
            loss.backward()
            if step % 50 == 0:
                print(f"Step {step}, Loss {loss.item()}")
            return loss

        optimizer.step(closure)

    return target
# Perform style transfer
output_img = run_style_transfer(content_image, style_image)

# Convert output image back to PIL format
def tensor_to_image(tensor):
    tensor = tensor.squeeze().cpu().detach().clamp(0, 1)
    tensor = transforms.ToPILImage()(tensor)
    return tensor

output_pil_image = tensor_to_image(output_img)
plt.imshow(output_pil_image)
plt.axis('off')
plt.show()

# Save or download the result
output_pil_image.save('output_image.png')
files.download('output_image.png')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: The size of tensor a (128) must match the size of tensor b (64) at non-singleton dimension 1